# **HW2 - Text Classification**

Name: `Mohammad Amin Ghasvari Jahromi`

Student Number: `97521432`

## 0. Imports

I Imported the dataset, numpy, and nltk packages.

In [1]:
# Imports
from keras.datasets import imdb
import numpy as np
import collections
from copy import deepcopy
import nltk
from nltk.corpus import stopwords
from nltk.stem import *
from nltk.util import ngrams
from nltk.util import ngrams
from nltk.tokenize import RegexpTokenizer


# nltk
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 1. IMDB Dataset

I tried to read from data set and print out some of these samples.

In [2]:
# Loading the dataset
print('Loading the IMDB dataset...')
(x_train, y_train), (x_test, y_test) = imdb.load_data()
print(f'Count train sequences: {len(x_train)}')
print(f'Count test sequences: {len(x_test)}')
print()

# Number of classes
print("Classes: ")
print(np.unique(y_train))
print()

# A Review example
print("Review-Example:")
print(x_train[0])
print()


Loading the IMDB dataset...
Count train sequences: 25000
Count test sequences: 25000

Classes: 
[0 1]

Review-Example:
[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16

## 2. Pre-Processing

### 2.1 Defining some useful variables and functions

Here are 2 functions. The first one `index_sen` is for converting index samples to word samples. On the other hand `sen_index` is for converting sentence samples to index samples.

In [3]:
word_index = imdb.get_word_index()
invtr_word_index = {i: word for word, i in word_index.items()}


def index_sen(iws, indexes_lists):
  """
  indexes_lists: list of arrays that contain indexes
  Returns lists of the sentence based on given indexes
  """

  sentences = []
  for indexes in indexes_lists:
    words = [iws.get(idx) for idx in indexes]
    sentences.append([word for word in words if word is not None])

  return np.array(sentences, dtype=object)

def sen_index(ws, sentences):
  """
  sentences: list of arrays that contain words
  Return lists of the indexes based on given sentence.
  """

  indexes_list = []
  for sentence in sentences:
    indexes = [ws.get(word) for word in sentence]
    indexes_list.append([index for index in indexes if index is not None])

  return np.array(indexes_list, dtype=object)
  
  
# Example for testing index_sen() and sen_index()
print(x_train[200])
print(index_sen(invtr_word_index, [x_train[200]]))
print(sen_index(word_index, [index_sen(invtr_word_index, [x_train[200]])[0]]))

[1, 14, 9, 6, 227, 196, 241, 634, 891, 234, 21, 12, 69, 6, 6, 176, 7, 4, 804, 4658, 2999, 667, 11, 12, 11, 85, 715, 6, 176, 7, 1565, 8, 1108, 10, 10, 12, 16, 1844, 78177, 33, 211, 21, 69, 49, 2009, 905, 388, 99, 17874, 125, 34, 6, 25904, 1274, 33, 4, 130, 7, 4, 22, 15, 16, 6424, 8, 650, 1069, 14, 22, 9, 44, 4609, 153, 154, 4, 318, 302, 1051, 23, 14, 22, 122, 6, 2093, 292, 10, 10, 723, 8721, 5, 11200, 9728, 71, 1344, 1576, 156, 11, 68, 251, 5, 36, 92, 4363, 133, 199, 743, 976, 354, 4, 64, 439, 9, 3059, 17, 32, 4, 12445, 26, 256, 34, 85847, 5, 49, 7, 98, 40, 2345, 9844, 43, 92, 168, 147, 474, 40, 8, 67, 6, 796, 97, 7, 14, 20, 19, 32, 2188, 156, 24, 18, 6090, 1007, 21, 8, 331, 97, 4, 65, 168, 5, 481, 53, 3084]
[['the' 'as' 'it' 'is' 'far' 'both' 'am' "i'll" 'unless' 'since' 'not'
  'that' 'me' 'is' 'is' 'quite' 'br' 'of' 'dialog' 'depicts' 'stupidity'
  'robert' 'this' 'that' 'this' 'because' 'jack' 'is' 'quite' 'br'
  'provides' 'in' 'island' 'i' 'i' 'that' 'with' 'plots' 'indicitive'
  

### 2.2 Decoding all train data

In [4]:
# Train sample 200 as a test
print(x_train[200])
print(x_test[200])

# Convert to real words
sentences = index_sen(invtr_word_index, x_train)
print(sentences[200])
test_sentences = index_sen(invtr_word_index, x_test)
print(test_sentences[200])

[1, 14, 9, 6, 227, 196, 241, 634, 891, 234, 21, 12, 69, 6, 6, 176, 7, 4, 804, 4658, 2999, 667, 11, 12, 11, 85, 715, 6, 176, 7, 1565, 8, 1108, 10, 10, 12, 16, 1844, 78177, 33, 211, 21, 69, 49, 2009, 905, 388, 99, 17874, 125, 34, 6, 25904, 1274, 33, 4, 130, 7, 4, 22, 15, 16, 6424, 8, 650, 1069, 14, 22, 9, 44, 4609, 153, 154, 4, 318, 302, 1051, 23, 14, 22, 122, 6, 2093, 292, 10, 10, 723, 8721, 5, 11200, 9728, 71, 1344, 1576, 156, 11, 68, 251, 5, 36, 92, 4363, 133, 199, 743, 976, 354, 4, 64, 439, 9, 3059, 17, 32, 4, 12445, 26, 256, 34, 85847, 5, 49, 7, 98, 40, 2345, 9844, 43, 92, 168, 147, 474, 40, 8, 67, 6, 796, 97, 7, 14, 20, 19, 32, 2188, 156, 24, 18, 6090, 1007, 21, 8, 331, 97, 4, 65, 168, 5, 481, 53, 3084]
[1, 92, 30, 4460, 14, 218, 246, 160, 1458, 463, 7, 4, 539, 39, 4172, 834, 4470, 8, 30, 4, 785, 16117, 768, 216, 8, 703, 8, 6585, 17, 76, 16769, 17, 614, 512, 151, 4, 3935, 203, 1467, 900, 14, 20, 9, 6, 1528, 35, 1557, 6823, 15, 47, 23464, 61, 113, 39, 4, 561, 13, 2984, 523, 725, 12,

### 2.3 Cleaning: Stopwords

I performed these kind of cleanings in this part.
- Remove the stopwords from the words
- Remove puncuations
- Remove numbers
- Remove words with length of < 2


In [5]:
# Apply stopwords cleaning
new_sentences = []
for sentence in sentences:
  tokenizer = RegexpTokenizer(r'\w+')
  tokenized = tokenizer.tokenize(' '.join(sentence))
  clean_stop_words = [word for word in tokenized if word not in stop_words and len(word) > 2]
  new_sentences.append(clean_stop_words)
# Test stopwords
print(new_sentences[200])


new_test_sentences = []
for sentence in test_sentences:
  tokenizer = RegexpTokenizer(r'\w+')
  tokenized = tokenizer.tokenize(' '.join(sentence))
  clean_stop_words = [word for word in tokenized if word not in stop_words and len(word) > 2]
  new_test_sentences.append(clean_stop_words)
# Test stopwords
print(new_test_sentences[200])

sentences = new_sentences
test_sentences = new_test_sentences

['far', 'unless', 'since', 'quite', 'dialog', 'depicts', 'stupidity', 'robert', 'jack', 'quite', 'provides', 'island', 'plots', 'indicitive', 'gets', 'good', 'shallow', 'earlier', 'understand', 'movies', 'suckers', 'better', 'tupac', 'catch', 'clash', 'change', 'cartoon', 'questionable', 'actors', 'work', 'excellent', 'instead', 'ended', 'comparison', 'together', 'whether', 'zeta', 'hitchhiker', 'forum', 'plan', 'tragic', 'hard', 'abysmal', 'scene', 'give', 'within', 'telling', 'boring', 'see', 'glass', 'movie', 'culprit', 'anyone', 'nananana', 'good', 'hanging', 'marvellous', 'final', 'sister', 'could', 'film', 'levels', 'regards', 'western', 'else', 'could', 'totally', 'spin']
['namely', 'interesting', 'worst', 'funny', 'lover', 'despite', 'decent', 'punk', 'dance', 'transfer', 'clear', 'sickeningly', 'ways', 'saw', 'cheap', 'freaky', 'movie', 'get', 'tactic', 'movie', 'shown', 'soon', 'old', 'explosion', 'action', 'include', 'let', 'americans', 'jean', 'pedestrian', 'housework', 'ac

### 2.4 Cleaning: Stemming

I used stemming to have better initial data.

In [6]:
# Apply stemming cleaning
new_sentences = []
st = PorterStemmer()
for sentence in sentences:
  clean_stemming = [st.stem(word) for word in sentence]
  new_sentences.append(clean_stemming)
# Test stemming
print(new_sentences[200])

sentences = new_sentences


# Apply stemming cleaning
new_test_sentences = []
st = PorterStemmer()
for sentence in test_sentences:
  clean_stemming = [st.stem(word) for word in sentence]
  new_test_sentences.append(clean_stemming)
# Test stemming
print(new_test_sentences[200])

test_sentences = new_test_sentences

['far', 'unless', 'sinc', 'quit', 'dialog', 'depict', 'stupid', 'robert', 'jack', 'quit', 'provid', 'island', 'plot', 'indicit', 'get', 'good', 'shallow', 'earlier', 'understand', 'movi', 'sucker', 'better', 'tupac', 'catch', 'clash', 'chang', 'cartoon', 'question', 'actor', 'work', 'excel', 'instead', 'end', 'comparison', 'togeth', 'whether', 'zeta', 'hitchhik', 'forum', 'plan', 'tragic', 'hard', 'abysm', 'scene', 'give', 'within', 'tell', 'bore', 'see', 'glass', 'movi', 'culprit', 'anyon', 'nananana', 'good', 'hang', 'marvel', 'final', 'sister', 'could', 'film', 'level', 'regard', 'western', 'els', 'could', 'total', 'spin']
['name', 'interest', 'worst', 'funni', 'lover', 'despit', 'decent', 'punk', 'danc', 'transfer', 'clear', 'sickeningli', 'way', 'saw', 'cheap', 'freaki', 'movi', 'get', 'tactic', 'movi', 'shown', 'soon', 'old', 'explos', 'action', 'includ', 'let', 'american', 'jean', 'pedestrian', 'housework', 'act', 'writer', 'seek', 'direct', 'move', 'start', 'chen', 'without', '

Then I needed to update my mappers, I mean `word_index` and `invrt_word_index`.

In [7]:
word_index = {tokenizer.tokenize(word)[0]: i for word, i in word_index.items() if len(word) > 2 and len(tokenizer.tokenize(word)) != 0}
invtr_word_index = {i: tokenizer.tokenize(word)[0] for word, i in word_index.items() if len(word) > 2 and len(tokenizer.tokenize(word)) != 0}

word_index = {st.stem(word): i for word, i in word_index.items()}
invtr_word_index = {i: st.stem(word) for word, i in word_index.items()}

9910


### 2.5 Convert and replace all train data

In [8]:
x_train = sen_index(word_index, sentences)
print(x_train[200])

x_test = sen_index(word_index, test_sentences)
print(x_test[200])

[79864, 891, 72270, 18399, 80925, 2820, 22992, 30465, 39159, 18399, 29976, 46060, 6516, 78177, 211, 39801, 17284, 905, 7729, 62986, 71633, 68513, 25904, 49676, 20584, 1440, 51005, 7530, 88284, 12218, 16082, 302, 4158, 5716, 32034, 723, 8721, 33228, 20563, 2461, 87492, 43774, 4363, 62134, 66156, 743, 713, 1097, 1082, 5096, 62986, 12445, 256, 85847, 39801, 46822, 9844, 51436, 33700, 50254, 1420, 48867, 2901, 16099, 32407, 50254, 71763, 12424]
[770, 925, 86059, 27562, 26235, 463, 539, 72039, 4200, 17505, 23339, 16117, 50682, 30241, 18900, 6585, 62986, 211, 38452, 62986, 614, 512, 30661, 6977, 83700, 1923, 3183, 87963, 1557, 39419, 23464, 914, 83414, 4843, 34510, 725, 39343, 10267, 85454, 85515, 82879, 51369, 30455, 6787, 23203, 25419, 14868, 5185, 11692, 86452, 79021, 7643, 85126, 285, 81891, 29182, 3209, 9678, 86059, 42268, 76123]


## 3. Build Models

### 3.1 Uni-Gram

In [9]:
positive_unigram = []
negative_unigram = []

# Length and order of these variables are the same
#   - sentences
#   - x_train
#   - y_train
for idx in range(len(x_train)):
  res = y_train[idx]
  unigrams = ngrams(deepcopy(sentences[idx]), 1)
  if res == 1:
    # It is positive comment
    positive_unigram.extend(unigrams)
  elif res == 0:
    # It is a negative comment
    negative_unigram.extend(unigrams)
  else: 
    raise Exception("undefined y for this dataset")

# For laplacian smoothing
positive_vocab_size = len(collections.Counter(positive_unigram))
negative_vocab_size = len(collections.Counter(negative_unigram))

# Lengths
len_positive_unigram = len(positive_unigram)
len_negative_unigram = len(negative_unigram)

# Laplacian smoothing
count_pos_unigram = dict(collections.Counter(positive_unigram))
count_neg_unigram = dict(collections.Counter(negative_unigram))

# <UNK> in .get()
unigram = {
  'pos': {v: (count_pos_unigram.get((k,), 0) + 1)/(len_positive_unigram + positive_vocab_size) for k, v in word_index.items()},
  'neg': {v: (count_neg_unigram.get((k,), 0) + 1)/(len_negative_unigram + negative_vocab_size) for k, v in word_index.items()}
}

#### 3.1.1 Evaluation

In [10]:
pp_initial = y_train[y_train==1].shape[0] / y_train.shape[0]
pn_initial = y_train[y_train==0].shape[0] / y_train.shape[0]

y_predict = []

for n in range(len(x_test)):
  w = x_test[n]
  pp = pp_initial * 10**300
  pn = pn_initial * 10**300

  for i in range(min(160, len(w))):
    pp *= unigram['pos'].get(w[i], 1)
    pn *= unigram['neg'].get(w[i], 1)
  y_predict.append(pp > pn)

In [11]:
tp = len([(idx, predict) for idx, predict in enumerate(y_predict) if y_test[idx] == True and predict == True])
fp = len([(idx, predict) for idx, predict in enumerate(y_predict) if y_test[idx] == False and predict == True])
fn = len([(idx, predict) for idx, predict in enumerate(y_predict) if y_test[idx] == True and predict == False])
tn = len([(idx, predict) for idx, predict in enumerate(y_predict) if y_test[idx] == False and predict == False])

accuracy = (tp + tn)/(tp + tn + fn + fp)
print("Accuracy: ", accuracy * 100)
precision = (tp)/(tp + fp)
print("Precision: ", precision * 100)
recall = (tp)/(tp + fn)
print("Recall: ", recall * 100)
print("F1-Score: ", (2 * precision * recall)/(precision + recall) * 100)

Accuracy:  79.384
Precision:  83.63553113553114
Recall:  73.064
F1-Score:  77.99316823228011


### 3.2. Bi-Gram


In [15]:
positive_bigram = []
negative_bigram = []

# Length and order of these variables are the same
#   - sentences
#   - x_train
#   - y_train
for idx in range(len(x_train)):
  res = y_train[idx]
  biigrams = ngrams(deepcopy(sentences[idx]), 2)
  if res == 1:
    # It is positive comment
    positive_bigram.extend(biigrams)
  elif res == 0:
    # It is a negative comment
    negative_bigram.extend(biigrams)
  else: 
    raise Exception("undefined y for this dataset")

# For laplacian smoothing
positive_vocab_size = len(collections.Counter(positive_bigram))
negative_vocab_size = len(collections.Counter(negative_bigram))

# Lengths
len_positive_bigram = len(positive_bigram)
len_negative_bigram = len(negative_bigram)

# Laplacian smoothing
count_pos_bigram = dict(collections.Counter(positive_bigram))
count_neg_bigram = dict(collections.Counter(negative_bigram))
count_all_bigram = set(dict(collections.Counter(positive_bigram)).keys()) | set(dict(collections.Counter(negative_bigram)).keys())

# <UNK> in .get()
bigram = {
  'pos': {(word_index.get(st.stem(k[0])), word_index.get(st.stem(k[1]))): (count_pos_bigram.get(k, 0) + 1)/(len_positive_bigram + len(count_all_bigram)) for k in count_all_bigram},
  'neg': {(word_index.get(st.stem(k[0])), word_index.get(st.stem(k[1]))): (count_neg_bigram.get(k, 0) + 1)/(len_negative_bigram + len(count_all_bigram)) for k in count_all_bigram}
}

#### 3.2.1 Evaluation

In [16]:
pp_initial = y_train[y_train==1].shape[0] / y_train.shape[0]
pn_initial = y_train[y_train==0].shape[0] / y_train.shape[0]

y_predict = []

for n in range(len(x_test)):
  w = x_test[n]
  pp = pp_initial * 10**300
  pn = pn_initial * 10**300

  for i in range(1, min(len(w), 150)):
    pp *= bigram['pos'].get((w[i-1], w[i]), 1)
    pn *= bigram['neg'].get((w[i-1], w[i]), 1)
  y_predict.append(pp > pn)

In [17]:
tp = len([(idx, predict) for idx, predict in enumerate(y_predict) if y_test[idx] == True and predict == True])
fp = len([(idx, predict) for idx, predict in enumerate(y_predict) if y_test[idx] == False and predict == True])
fn = len([(idx, predict) for idx, predict in enumerate(y_predict) if y_test[idx] == True and predict == False])
tn = len([(idx, predict) for idx, predict in enumerate(y_predict) if y_test[idx] == False and predict == False])

accuracy = (tp + tn)/(tp + tn + fn + fp)
print("Accuracy: ", accuracy * 100)
precision = (tp)/(tp + fp)
print("Precision: ", precision * 100)
recall = (tp)/(tp + fn)
print("Recall: ", recall * 100)
print("F1-Score: ", (2 * precision * recall)/(precision + recall) * 100)

Accuracy:  80.536
Precision:  85.18620943952803
Recall:  73.92800000000001
F1-Score:  79.15881445948261


### 3.3. Tri-Gram

In [21]:
positive_trigram = []
negative_trigram = []

# Length and order of these variables are the same
#   - sentences
#   - x_train
#   - y_train
for idx in range(len(x_train)):
  res = y_train[idx]
  trigrams = ngrams(deepcopy(sentences[idx]), 3)
  if res == 1:
    # It is positive comment
    positive_trigram.extend(trigrams)
  elif res == 0:
    # It is a negative comment
    negative_trigram.extend(trigrams)
  else: 
    raise Exception("undefined y for this dataset")

# For laplacian smoothing
positive_vocab_size = len(collections.Counter(positive_trigram))
negative_vocab_size = len(collections.Counter(negative_trigram))

# Lengths
len_positive_trigram = len(positive_trigram)
len_negative_trigram = len(negative_trigram)

# Laplacian smoothing
count_pos_trigram = dict(collections.Counter(positive_trigram))
count_neg_trigram = dict(collections.Counter(negative_trigram))
count_all_trigram = set(dict(collections.Counter(positive_trigram)).keys()) | set(dict(collections.Counter(negative_trigram)).keys())

# <UNK> in .get()
trigram = {
  'pos': {(word_index.get(st.stem(k[0])), word_index.get(st.stem(k[1])), word_index.get(st.stem(k[2]))): (count_pos_bigram.get(k, 0) + 1)/(len_positive_trigram + len(count_all_trigram)) for k in count_all_trigram},
  'neg': {(word_index.get(st.stem(k[0])), word_index.get(st.stem(k[1])), word_index.get(st.stem(k[2]))): (count_neg_bigram.get(k, 0) + 1)/(len_negative_trigram + len(count_all_trigram)) for k in count_all_trigram}
}

#### 3.3.1 Evaluation

In [20]:
pp_initial = y_train[y_train==1].shape[0] / y_train.shape[0]
pn_initial = y_train[y_train==0].shape[0] / y_train.shape[0]

y_predict = []

for n in range(len(x_test)):
  w = x_test[n]
  pp = pp_initial * 10**300
  pn = pn_initial * 10**300

  for i in range(2, min(len(w), 160)):
    pt = trigram['pos'].get((w[i-2], w[i-1], w[i]), 1)
    pneg = trigram['neg'].get((w[i-2], w[i-1], w[i]), 1)

    if pt == 1 or pneg == 1:
      pt = 1
      pneg = 1
      a = sum([bigram['pos'].get((w[i-2], w[i-1]), 1), bigram['pos'].get((w[i-2], w[i]), 1), bigram['pos'].get((w[i-1], w[i]), 1)])/3
      b = sum([bigram['neg'].get((w[i-2], w[i-1]), 1), bigram['neg'].get((w[i-2], w[i]), 1), bigram['neg'].get((w[i-1], w[i]), 1)])/3
      if a != 1 and b != 1:
        pt = a
        pneg = b


    pp *= pt
    pn *= pneg

  y_predict.append(pp > pn)

In [22]:
tp = len([(idx, predict) for idx, predict in enumerate(y_predict) if y_test[idx] == True and predict == True])
fp = len([(idx, predict) for idx, predict in enumerate(y_predict) if y_test[idx] == False and predict == True])
fn = len([(idx, predict) for idx, predict in enumerate(y_predict) if y_test[idx] == True and predict == False])
tn = len([(idx, predict) for idx, predict in enumerate(y_predict) if y_test[idx] == False and predict == False])

accuracy = (tp + tn)/(tp + tn + fn + fp)
print("Accuracy: ", accuracy * 100)
precision = (tp)/(tp + fp)
print("Precision: ", precision * 100)
recall = (tp)/(tp + fn)
print("Recall: ", recall * 100)
print("F1-Score: ", (2 * precision * recall)/(precision + recall) * 100)

Accuracy:  74.896
Precision:  81.58108382382788
Recall:  64.312
F1-Score:  71.9244877874206


## 4. Questions

- I needed to multiply the probabilities to a large number because they became 0.
- Yes we can see bag of words effect from the results.
- I ran this model multiple time. I added stemming to have one single token for the same words. I removed the puncuation. I removed the number.
- I got out of memory errror, so i had to do some optmization to ignore some considerations.
- The notable number of tokens in test sentences are not available in my dictionay that caused some errors in the results. 
- The bigram turns out to be the best choice for classification and then unigram.

## Good Luck!